In [1]:
import numpy as np
import pandas as pd

# Procesamos el Dataset en un Dataframe de Pandas

In [2]:
dataset = pd.read_excel("./src/shuttle-landing-control.ods")

print(dataset)

   Stability Error Sign  Wind   Magnitude Visibility   Class
0          *     *    *     *           *        yes    auto
1      xstab     *    *     *           *         no  noauto
2       stab    lx    *     *           *         no  noauto
3       stab    xl    *     *           *         no  noauto
4       stab    mm   nn  tail           *         no  noauto
5          *     *    *     *  OutOfRange         no  noauto
6       stab    ss    *     *         low         no    auto
7       stab    ss    *     *      medium         no    auto
8       stab    ss    *     *      strong         no    auto
9       stab    mm   pp  head         low         no    auto
10      stab    mm   pp  head      medium         no    auto
11      stab    mm   pp  tail         low         no    auto
12      stab    mm   pp  tail      medium         no    auto
13      stab    mm   pp  head      strong         no  noauto
14      stab    mm   pp  tail      strong         no    auto


### Antes de pasar a la metodologìa STAR damos tratamiento a los casos don't care (*)

In [3]:
from librerias_caseras.procesamiento_datasets import valores_observados, repeticiones_de_valor
import itertools    # <- https://blog.enterprisedna.co/how-to-generate-all-combinations-of-a-list-in-python/#:~:text=To%20generate%20all%20combinations%20of%20a%20list%20in%20Python%2C%20you,iterator%20algebra%20and%20combinatorial%20constructs.

def expandirTabla_desde_fila(fila:list[str], etiquetas_columnas:list[str], dataset:pd.DataFrame):
    '''Evalùa una fila del conjunto de datos, y si encuentra un * expande la tabla con todos los casos del dominio del atributo'''

    se_expande:bool = False
    contador:int = 0
    indices_de_expansion:list = []
    indices_ordinarios:list = []

    for i, valor_observado in enumerate(fila):
        if(valor_observado == '*'):
            se_expande = True
            contador += 1
            indices_de_expansion.append(i)
        else:
            contador += 1
            indices_ordinarios.append(i)

    if se_expande:

        dominios_atributos:list[list[str]] = []
        # observamos el dominio posible del atributo (en el preanálisis o bservamos que todos los valores del dominio de cada atributo aparece en su columna)
        for indice in range(len(fila)):

            if(indices_de_expansion.__contains__(indice)):

                dominios_atributos.append(valores_observados(valores_observados=dataset[etiquetas_columnas[indice]]))

            else:

                dominios_atributos.append([fila[indice]])

        print(dominios_atributos)
        
        for dominio_atributo in dominios_atributos:
            print(dominio_atributo)

            if(dominio_atributo.__contains__('*')):
                dominio_atributo.pop(dominio_atributo.index('*'))

        #ahora que tenemos todos los dominios de atributos, para ver que funciona, vamos a sacar todas las conbinaciones posibles con esos dimnios

        combinaciones:list[list[str]] = []

        for combinacion in itertools.product(*dominios_atributos):       #<- https://stackoverflow.com/questions/533905/how-to-get-the-cartesian-product-of-multiple-lists

            # print("\n\nlas conbinaciones de esos atributos: \n\n\n",combinacion)
            combinaciones.append(list(combinacion))

        print("\n\nlas conbinaciones de esos atributos: \n\n\n", np.matrix(combinaciones))

        return combinaciones
    
    else:

        return [fila]



In [4]:
filas_nuevo_dataset:list[list[str]] = []

# for row in dataset[dataset['Visibility'] == 'yes'].values.tolist():
for row in dataset.values.tolist():
    expansion_fila = expandirTabla_desde_fila(fila=row, etiquetas_columnas=dataset.columns.to_list(), dataset=dataset)

    for fila in expansion_fila:
        filas_nuevo_dataset.append(fila)

print("\n\nesto es lo que se contiene enla lista de filas nuevo dataset: \n\n", np.matrix(filas_nuevo_dataset))

[['*', 'xstab', 'stab'], ['*', 'lx', 'xl', 'mm', 'ss'], ['*', 'nn', 'pp'], ['*', 'tail', 'head'], ['*', 'OutOfRange', 'low', 'medium', 'strong'], ['yes'], ['auto']]
['*', 'xstab', 'stab']
['*', 'lx', 'xl', 'mm', 'ss']
['*', 'nn', 'pp']
['*', 'tail', 'head']
['*', 'OutOfRange', 'low', 'medium', 'strong']
['yes']
['auto']


las conbinaciones de esos atributos: 


 [['xstab' 'lx' 'nn' 'tail' 'OutOfRange' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'tail' 'low' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'tail' 'medium' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'tail' 'strong' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'head' 'OutOfRange' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'head' 'low' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'head' 'medium' 'yes' 'auto']
 ['xstab' 'lx' 'nn' 'head' 'strong' 'yes' 'auto']
 ['xstab' 'lx' 'pp' 'tail' 'OutOfRange' 'yes' 'auto']
 ['xstab' 'lx' 'pp' 'tail' 'low' 'yes' 'auto']
 ['xstab' 'lx' 'pp' 'tail' 'medium' 'yes' 'auto']
 ['xstab' 'lx' 'pp' 'tail' 'strong' 'yes' 'auto']
 ['xstab' 'lx' 'pp' 'head' 'OutO

### una vez que tenemos las filas de lo que sería la tabla expandida, procedemos a construir el nuevo dataframe con la información desarrollada

In [5]:
nuevo_dataset = pd.DataFrame(data=filas_nuevo_dataset, columns=dataset.columns)

print(nuevo_dataset)

    Stability Error Sign  Wind   Magnitude Visibility   Class
0       xstab    lx   nn  tail  OutOfRange        yes    auto
1       xstab    lx   nn  tail         low        yes    auto
2       xstab    lx   nn  tail      medium        yes    auto
3       xstab    lx   nn  tail      strong        yes    auto
4       xstab    lx   nn  head  OutOfRange        yes    auto
..        ...   ...  ...   ...         ...        ...     ...
273      stab    mm   pp  head      medium         no    auto
274      stab    mm   pp  tail         low         no    auto
275      stab    mm   pp  tail      medium         no    auto
276      stab    mm   pp  head      strong         no  noauto
277      stab    mm   pp  tail      strong         no    auto

[278 rows x 7 columns]


# Pasos de la metodologìa STAR

## Particionar la muestra en casos positivos y negativos

In [6]:
casos_positivos = nuevo_dataset[nuevo_dataset["Class"] == 'auto']

casos_negativos = nuevo_dataset[nuevo_dataset['Class'] == 'noauto']


print("casos positivos: \n\n", casos_positivos)
print("casos negativos: \n\n", casos_negativos)

casos positivos: 

     Stability Error Sign  Wind   Magnitude Visibility Class
0       xstab    lx   nn  tail  OutOfRange        yes  auto
1       xstab    lx   nn  tail         low        yes  auto
2       xstab    lx   nn  tail      medium        yes  auto
3       xstab    lx   nn  tail      strong        yes  auto
4       xstab    lx   nn  head  OutOfRange        yes  auto
..        ...   ...  ...   ...         ...        ...   ...
272      stab    mm   pp  head         low         no  auto
273      stab    mm   pp  head      medium         no  auto
274      stab    mm   pp  tail         low         no  auto
275      stab    mm   pp  tail      medium         no  auto
277      stab    mm   pp  tail      strong         no  auto

[145 rows x 7 columns]
casos negativos: 

     Stability Error Sign  Wind   Magnitude Visibility   Class
128     xstab    lx   nn  tail  OutOfRange         no  noauto
129     xstab    lx   nn  tail         low         no  noauto
130     xstab    lx   nn  tail

## Seleccionar aleatoriamente un caso positivo

In [7]:

semilla = casos_positivos.sample()  # <- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html


print(semilla)

    Stability Error Sign  Wind Magnitude Visibility Class
122      stab    ss   pp  tail    medium        yes  auto


## Generar la estrella que cubra a la semilla

### pero primero, vamos a obtener todos los dominios de cada columna, porque los vamos a necesitar a continuacion

In [8]:
dominios_completos_atributos:list[list[str]] = []

for atributo in dataset.columns.to_list():

    dominios_completos_atributos.append(valores_observados(dataset[atributo].to_list()))

### ahora sí generamos la estrella

In [9]:
from star import estrella

una_estrella = estrella()

una_estrella.generarEstrella(semilla, dominios_completos_atributos)

print(una_estrella.Su_Cubrimiento)

[['stab', 'ss', 'pp', 'tail', 'medium', 'yes']]
las combinaciones son: 

 [['stab'], ['ss'], ['pp'], ['tail'], ['medium'], ['yes'], ['stab', 'ss'], ['stab', 'pp'], ['stab', 'tail'], ['stab', 'medium'], ['stab', 'yes'], ['ss', 'pp'], ['ss', 'tail'], ['ss', 'medium'], ['ss', 'yes'], ['pp', 'tail'], ['pp', 'medium'], ['pp', 'yes'], ['tail', 'medium'], ['tail', 'yes'], ['medium', 'yes'], ['stab', 'ss', 'pp'], ['stab', 'ss', 'tail'], ['stab', 'ss', 'medium'], ['stab', 'ss', 'yes'], ['stab', 'pp', 'tail'], ['stab', 'pp', 'medium'], ['stab', 'pp', 'yes'], ['stab', 'tail', 'medium'], ['stab', 'tail', 'yes'], ['stab', 'medium', 'yes'], ['ss', 'pp', 'tail'], ['ss', 'pp', 'medium'], ['ss', 'pp', 'yes'], ['ss', 'tail', 'medium'], ['ss', 'tail', 'yes'], ['ss', 'medium', 'yes'], ['pp', 'tail', 'medium'], ['pp', 'tail', 'yes'], ['pp', 'medium', 'yes'], ['tail', 'medium', 'yes'], ['stab', 'ss', 'pp', 'tail'], ['stab', 'ss', 'pp', 'medium'], ['stab', 'ss', 'pp', 'yes'], ['stab', 'ss', 'tail', 'medium']

## Eliminar complejos que cubren ejemplos negativos

In [10]:
una_estrella.eliminar_complejos_que_cubren_ejemplos_negativos(casos_negativos)

print(una_estrella.Su_Cubrimiento)

la etiqueta:  complejo 3
la etiqueta:  complejo 4
la etiqueta:  complejo 18
la etiqueta:  complejo 2
la etiqueta:  complejo 15
la etiqueta:  complejo 16
la etiqueta:  complejo 37
la etiqueta:  complejo 1
la etiqueta:  complejo 12
la etiqueta:  complejo 13
la etiqueta:  complejo 34
la etiqueta:  complejo 11
la etiqueta:  complejo 31
la etiqueta:  complejo 32
la etiqueta:  complejo 51
la etiqueta:  complejo 0
la etiqueta:  complejo 8
la etiqueta:  complejo 9
la etiqueta:  complejo 28
la etiqueta:  complejo 7
la etiqueta:  complejo 25
la etiqueta:  complejo 26
la etiqueta:  complejo 47
la etiqueta:  complejo 6
la etiqueta:  complejo 22
la etiqueta:  complejo 21
la etiqueta:  complejo 41
cubrimiento: cubrimiento de ['stab', 'ss', 'pp', 'tail', 'medium', 'yes'] -> ['auto']

complejo: complejo 5

[Visibility = yes] 


complejo: complejo 10

[Stability = stab] [Visibility = yes] 


complejo: complejo 14

[Error = ss] [Visibility = yes] 


complejo: complejo 17

[Sign = pp] [Visibility = yes] 

## Reducir la estrella en la medida de lo posible